In [1]:
import pandas as pd
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import teradatasql
import numpy as np
import os

- Открываем файл.
- Импортируем данные.
- Обрабатываем данные.
- Делаем бэкап UAT_PRODUCT.PRODUCT_PARAMETERS в UAT_PRODUCT.PRODUCT_PARAMETERS_BACKUP.
- Грузим обработанные данные в UAT_PRODUCT.PRODUCT_PARAMETERS_TEST_FOR_SAP.
- Запросом транформируем данные в подневные из UAT_PRODUCT.PRODUCT_PARAMETERS_TEST_FOR_SAP в UAT_PRODUCT.PRODUCT_PARAMETERS.

In [2]:
class SAP():
    
    def __init__(self, path=r'P:\\CP_PLM\\Reporting\\Report_Data\\Revenue & Subs\\'):
        self.path = path
        self.shortname = None
        self.condition = str()
        self.choose_file()
        self.raw = pd.DataFrame()
        
 
    def choose_file(self):
        for file in enumerate(os.listdir(self.path)):
            print(file)
        while not self.shortname:
            file_index = int(input('Какой номер нужен? '))
            for i, file in enumerate(os.listdir(self.path)):
                if i == file_index:
                    print(f'Выбран {file}')
                    self.shortname = file
                    self.filename = self.path + self.shortname
                    break
                
            
    def import_file(self, filename = None):
        '''
        как результат - получаем self.raw
        '''
        if not filename: filename = self.filename
        dtype_dict = {
            'version': str, 'month': str, 'year': str, 'region': str, 'base_type': str, 
            'tariff': str, 'account': str, 'param_value': float
        }
        
        header_dict = {
            'data' : ['version','month','year','region','account','param_value'],
            'phd' : ['version','month','year','region','base_type','tariff','account','param_value'],
            'rev' : ['version','month','year','region','base_type','tariff','account','param_value']
        }
        
        for sh_name in ['Region']:
            try:
                sh_df = pd.read_excel(filename, sheet_name = sh_name, engine='pyxlsb',
                    names = header_dict[sh_name],
                    usecols = header_dict[sh_name],
                    dtype = dtype_dict, skiprows = 0)
            except:
                continue
            else:
                sh_df = sh_df.dropna()
                self.raw = pd.concat([self.raw,sh_df])
        print(self.raw.head())
        
        
        
    def apply_condition(self, param_dict = None):
        '''
        применяем условия к датафрейму и как результат - получаем self.df для инсерта
        '''
        if not param_dict: param_dict = dict()
            
        if param_dict.get('min_date', None):
            min_date_dt = datetime.strptime(param_dict['min_date'], '%Y-%m-%d').date()
        else:
            min_date_dt = self.full_df.report_month.min()
        min_date_str = min_date_dt.strftime('%Y-%m-%d')
            
            
        if param_dict.get('max_date', None):
            max_date_dt = datetime.strptime(param_dict['max_date'], '%Y-%m-%d').date()
        else:
            max_date_dt = self.full_df.report_month.max()
        max_date_dt = (max_date_dt + relativedelta(months=1)).replace(day=1) - relativedelta(days=1)
        max_date_str = max_date_dt.strftime('%Y-%m-%d')
        
        
        
        if param_dict.get('param_1', None):
            param_1 = param_dict['param_1']
        else:
            param_1 = [param.strip() for param in self.full_df.version.unique()]
            
        if param_dict.get('param_2', None):
            param_2 = param_dict['param_2']
        else:
            param_2 = [param.strip() for param in self.full_df.account.unique()]
            
            
        self.df = self.full_df[
                (self.full_df.report_month >= min_date_dt)
                & (self.full_df.report_month <= max_date_dt)
                & (self.full_df.version.isin(param_1))
                & (self.full_df.account.isin(param_2))
        ]
        cols = ['report_month', 'version', 'account','NULL', 'NULL1', 'BRANCH_ID', 'base_type',
                'NULL2', 'NULL3', 'tariff', 'NULL4', 'param_value']
        self.df = self.df.reindex(columns = cols,  fill_value = np.NaN).where((pd.notnull(self.df)), None)
        
        
        '''
        формулируем те же условия в SQL, чтобы затирать при необходимости эти же данные в терадате
        '''
        self.condition = f'''
        WHERE REPORT_DATE BETWEEN DATE'{min_date_str}' AND DATE'{max_date_str}'
        AND PARAM_1 in ({','.join("'{0}'".format(p) for p in param_1)}) 
        AND PARAM_2 in ({','.join("'{0}'".format(p) for p in param_2)})
        '''

        self.uat_pp_df = self.get_uat_pp_df()
        print(self.condition)
        
    
    def get_sap_codes(self):
        with teradatasql.connect() as session:
            query = '''
                    sel branch_id, sap_code, sap_name_ru
                    FROM PRD2_DIC_V.BRANCH
                    where SAP_CODE is not null
                    '''
            return pd.read_sql(query, session)
                
    
    def process_df(self):
        '''
        как результат - получаем self.full_df
        '''
        
        self.full_df = self.raw.copy()
        self.full_df['report_month'] = self.full_df.apply(lambda row: date(int(row.year), int(row.month), 1), axis = 1)
        self.full_df.param_value = self.full_df.param_value.apply(lambda x: round(x,4))
        
        if 'tariff' in self.full_df:
            tariff_dict = {'1100' : 'Bundle', '2000' : 'PAYG'}
            self.full_df.tariff = self.full_df.tariff.replace(tariff_dict)
            self.full_df.tariff = self.full_df.tariff.replace('#','NO SUBS')
        
        if 'base_type' in self.full_df:
            self.full_df.base_type = self.full_df.base_type.replace('#','NOT A')
        
        self.full_df.version = self.full_df.version.apply(lambda x: 'BU' if 'BU' in x else x.strip())
        self.sap_codes_df = self.get_sap_codes()
        self.full_df = pd.merge(self.full_df, self.sap_codes_df, how='inner', left_on='region', right_on='SAP_CODE')       
        print(self.full_df.head())
        
    
    def make_uat_pp_backup(self):
        with teradatasql.connect() as session:
            with session.cursor() as cur:
                cur.execute('''
                    DELETE FROM UAT_PRODUCT.PRODUCT_PARAMETERS_BACKUP; 
                    INSERT INTO UAT_PRODUCT.PRODUCT_PARAMETERS_BACKUP
                    SELECT * FROM UAT_PRODUCT.PRODUCT_PARAMETERS 
                ''')
        print(f'{cur.rowcount} rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS_BACKUP.')
        
    
    def get_uat_pp_df(self):
        with teradatasql.connect() as session:
            query = 'sel * from UAT_PRODUCT.PRODUCT_PARAMETERS ' + self.condition
            return pd.read_sql(query, session)
            
            
    def get_df_diff(self, df1, df2, which='right_only'):
        comparison_df = df1.merge(df2,
                                  indicator=True,
                                  how='outer')
        if which is None:
            diff_df = comparison_df[comparison_df['_merge'] != 'both']
        else:
            diff_df = comparison_df[comparison_df['_merge'] == which]
        return diff_df
    
    
    def update_uat_pp_test(self, batchsize = 100000):
        with teradatasql.connect() as session:
            with session.cursor() as cur:
                print('deleting from UAT_PRODUCT.PRODUCT_PARAMETERS_TEST_FOR_SAP...')
                cur.execute('''delete from UAT_PRODUCT.PRODUCT_PARAMETERS_TEST_FOR_SAP;''')
                print(f'{cur.rowcount} rows deleted from UAT_PRODUCT.PRODUCT_PARAMETERS_TEST_FOR_SAP\n')

                print('inserting into UAT_PRODUCT.PRODUCT_PARAMETERS_TEST_FOR_SAP...')
                for num in range(0, len(self.df), batchsize):
                    cur.executemany(f'''
                     INSERT into UAT_PRODUCT.PRODUCT_PARAMETERS_TEST_FOR_SAP ({','.join('?'*len(self.df.columns))})
                    ''',
                        [tuple(row) for row in self.df.iloc[num:num+batchsize,:].itertuples(index=False)]
                        )
                print(f'{len(self.df)} rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS_TEST_FOR_SAP.')
                
                
    def delete_from_uat_pp(self):
        query = 'DELETE from UAT_PRODUCT.PRODUCT_PARAMETERS ' + self.condition      
        with teradatasql.connect() as session:
            with session.cursor() as cur:
                cur.execute(query)      
                print(f'{cur.rowcount} rows deleted from UAT_PRODUCT.PRODUCT_PARAMETERS')
                
                
    def insert_into_uat_pp(self, replace=True):
        self.update_uat_pp_test()
        self.make_uat_pp_backup()
        if replace: self.delete_from_uat_pp()

        query = '''
            insert into UAT_PRODUCT.PRODUCT_PARAMETERS
            sel
                cal.calendar_date as REPORT_DATE, test.PARAM_1, test.PARAM_2, test.PARAM_3,
                test.PARAM_4, test.BRANCH_ID, test.BASE_TYPE, test.TP_ID_1, test.TP_ID_2,
                test.TARIFF_1, test.TARIFF_2, PARAM_VALUE/EXTRACT(DAY FROM LAST_DAY(REPORT_DATE)) as PARAM_VALUE
             from uat_product.product_parameters_test_for_sap test
             left join Sys_Calendar.BusinessCalendar cal
                on test.REPORT_DATE = trunc(cal.calendar_date,'mon')
             where test.PARAM_1 <> 'AC'
             union all
             sel *
             from uat_product.product_parameters_test_for_sap test
             where test.PARAM_1 = 'AC'
        '''
                
        with teradatasql.connect() as session:
            with session.cursor() as cur:
                cur.execute(query)      
                print(f'{cur.rowcount} rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS.')

In [3]:
df = pd.read_excel(r'P:\\CP_PLM\\Reporting\\Report_Data\\Revenue & Subs\\Core Live Subs BU.xlsx', sheet_name = 'Region')

In [4]:
df.columns = ['Cluster', 'KPI', 'Код региона', 'Регион', 'Min', 'Target','Stretch']
df.head()

,Cluster,KPI,Код региона,Регион,Min,Target,Stretch
0,Challenger Elite,Core Live Subs,2330,Tele2 - Vladimir,290024.0,300516.0,308639.0
1,Challenger wo SPB,Core Live Subs,2400,Tele2 - Kaluga,184342.0,192558.0,198281.0
2,Challenger wo SPB,Core Live Subs,2440,Tele2 - Kostroma,86525.0,90652.0,93312.0
3,Challenger wo SPB,Core Live Subs,2620,Tele2 - Ryazan,192061.0,200141.0,206126.0
4,Challenger Elite,Core Live Subs,2670,Tele2 - Smolensk,197808.0,211166.0,217501.0


In [5]:
df = df[df['KPI'] == 'Core Live Subs']
df.head()

,Cluster,KPI,Код региона,Регион,Min,Target,Stretch
0,Challenger Elite,Core Live Subs,2330,Tele2 - Vladimir,290024.0,300516.0,308639.0
1,Challenger wo SPB,Core Live Subs,2400,Tele2 - Kaluga,184342.0,192558.0,198281.0
2,Challenger wo SPB,Core Live Subs,2440,Tele2 - Kostroma,86525.0,90652.0,93312.0
3,Challenger wo SPB,Core Live Subs,2620,Tele2 - Ryazan,192061.0,200141.0,206126.0
4,Challenger Elite,Core Live Subs,2670,Tele2 - Smolensk,197808.0,211166.0,217501.0


In [6]:
df['REPORT_DATE'] = '2021-01-01'
df.head()

,Cluster,KPI,Код региона,Регион,Min,Target,Stretch,REPORT_DATE
0,Challenger Elite,Core Live Subs,2330,Tele2 - Vladimir,290024.0,300516.0,308639.0,2021-01-01
1,Challenger wo SPB,Core Live Subs,2400,Tele2 - Kaluga,184342.0,192558.0,198281.0,2021-01-01
2,Challenger wo SPB,Core Live Subs,2440,Tele2 - Kostroma,86525.0,90652.0,93312.0,2021-01-01
3,Challenger wo SPB,Core Live Subs,2620,Tele2 - Ryazan,192061.0,200141.0,206126.0,2021-01-01
4,Challenger Elite,Core Live Subs,2670,Tele2 - Smolensk,197808.0,211166.0,217501.0,2021-01-01


In [7]:
df.head()

,Cluster,KPI,Код региона,Регион,Min,Target,Stretch,REPORT_DATE
0,Challenger Elite,Core Live Subs,2330,Tele2 - Vladimir,290024.0,300516.0,308639.0,2021-01-01
1,Challenger wo SPB,Core Live Subs,2400,Tele2 - Kaluga,184342.0,192558.0,198281.0,2021-01-01
2,Challenger wo SPB,Core Live Subs,2440,Tele2 - Kostroma,86525.0,90652.0,93312.0,2021-01-01
3,Challenger wo SPB,Core Live Subs,2620,Tele2 - Ryazan,192061.0,200141.0,206126.0,2021-01-01
4,Challenger Elite,Core Live Subs,2670,Tele2 - Smolensk,197808.0,211166.0,217501.0,2021-01-01


In [8]:

with teradatasql.connect() as session:
    query = '''
            sel branch_id, sap_code, sap_name_ru
            FROM PRD2_DIC_V.BRANCH
            where SAP_CODE is not null
            '''
    sap_codes = pd.read_sql(query, session)
sap_codes.head()

,BRANCH_ID,SAP_CODE,SAP_NAME_RU
0,15.0,248000,Tele2-Липецк
1,55.0,256000,Tele2-Оренбург
2,106.0,212000,Tele2-Марий Эл
3,36.0,335000,Tele2-Вологда
4,100.0,286000,Tele2 - ХМАО


In [9]:
sap_codes.SAP_CODE.unique()

array(['248000', '256000', '212000', '335000', '286000', '246000',
       '221000', '203000', '252000', '271000', '265000', '219000',
       '274000', '242000', '239000', '245000', '224000', '238000',
       '218000', '204000', '310000', '234000', '267000', '257000',
       '249000', '258000', '241000', 'X25000', '378000', '223000',
       '232000', '269000', '253000', '268000', '213000', '233000',
       '229000', '277000', '270000', '240000', '276000', '272000',
       '237000', '211000', '216000', '243000', '244000', '222000',
       '236000', '289000', '254000', '278000', '251000', '231000',
       '263000', '264000', '360000', '279000', '262000', '273000',
       '261000', '266000', '255000', '259000'], dtype=object)

In [10]:
df_old = df.copy()
df['Код региона'] = df['Код региона'].apply(lambda x: str(x) + '00')
df.head()

,Cluster,KPI,Код региона,Регион,Min,Target,Stretch,REPORT_DATE
0,Challenger Elite,Core Live Subs,233000,Tele2 - Vladimir,290024.0,300516.0,308639.0,2021-01-01
1,Challenger wo SPB,Core Live Subs,240000,Tele2 - Kaluga,184342.0,192558.0,198281.0,2021-01-01
2,Challenger wo SPB,Core Live Subs,244000,Tele2 - Kostroma,86525.0,90652.0,93312.0,2021-01-01
3,Challenger wo SPB,Core Live Subs,262000,Tele2 - Ryazan,192061.0,200141.0,206126.0,2021-01-01
4,Challenger Elite,Core Live Subs,267000,Tele2 - Smolensk,197808.0,211166.0,217501.0,2021-01-01


In [11]:
df['Код региона'].unique()

array(['233000', '240000', '244000', '262000', '267000', '269000',
       '271000', '237000', '276000', '231000', '232000', '236000',
       '246000', '248000', '257000', '258000', '213000', '264000',
       '268000', '279000', 'X25000', '238000', '241000', '249000',
       '265000', '203000', '277000', '219000', '222000', '242000',
       '224000', '254000', '255000', '270000', '204000', '234000',
       '223000', '261000', '266000', '245000', '256000', '259000',
       '289000', '211000', '272000', '286000', '274000', '218000',
       '212000', '216000', '243000', '252000', '263000', '273000',
       '221000', '229000', '253000', '335000', '239000', '251000',
       '310000', '360000', '378000'], dtype=object)

In [12]:
df_old = df.copy()
df = pd.merge(df, sap_codes, how='inner', left_on='Код региона', right_on='SAP_CODE')
df.head()

,Cluster,KPI,Код региона,Регион,Min,Target,Stretch,REPORT_DATE,BRANCH_ID,SAP_CODE,SAP_NAME_RU
0,Challenger Elite,Core Live Subs,233000,Tele2 - Vladimir,290024.0,300516.0,308639.0,2021-01-01,28.0,233000,Tele2-Владимир
1,Challenger wo SPB,Core Live Subs,240000,Tele2 - Kaluga,184342.0,192558.0,198281.0,2021-01-01,31.0,240000,Tele2-Калуга
2,Challenger wo SPB,Core Live Subs,244000,Tele2 - Kostroma,86525.0,90652.0,93312.0,2021-01-01,32.0,244000,Tele2-Кострома
3,Challenger wo SPB,Core Live Subs,262000,Tele2 - Ryazan,192061.0,200141.0,206126.0,2021-01-01,39.0,262000,Tele2-Рязань
4,Challenger Elite,Core Live Subs,267000,Tele2 - Smolensk,197808.0,211166.0,217501.0,2021-01-01,11.0,267000,Tele2-Смоленск


In [13]:
df['PARAM_1'] = 'BU'
df.head()

,Cluster,KPI,Код региона,Регион,Min,Target,Stretch,REPORT_DATE,BRANCH_ID,SAP_CODE,SAP_NAME_RU,PARAM_1
0,Challenger Elite,Core Live Subs,233000,Tele2 - Vladimir,290024.0,300516.0,308639.0,2021-01-01,28.0,233000,Tele2-Владимир,BU
1,Challenger wo SPB,Core Live Subs,240000,Tele2 - Kaluga,184342.0,192558.0,198281.0,2021-01-01,31.0,240000,Tele2-Калуга,BU
2,Challenger wo SPB,Core Live Subs,244000,Tele2 - Kostroma,86525.0,90652.0,93312.0,2021-01-01,32.0,244000,Tele2-Кострома,BU
3,Challenger wo SPB,Core Live Subs,262000,Tele2 - Ryazan,192061.0,200141.0,206126.0,2021-01-01,39.0,262000,Tele2-Рязань,BU
4,Challenger Elite,Core Live Subs,267000,Tele2 - Smolensk,197808.0,211166.0,217501.0,2021-01-01,11.0,267000,Tele2-Смоленск,BU


In [14]:
df_old = df.copy()
df = df.melt(id_vars=['Cluster', 'KPI', 'Код региона', 'Регион','REPORT_DATE', 'BRANCH_ID', 'SAP_CODE', 'SAP_NAME_RU', 'PARAM_1'],
                 value_vars=['Min', 'Target', 'Stretch'],
                 var_name='PARAM_3',
                 value_name='PARAM_VALUE'
                 )
df.head()

,Cluster,KPI,Код региона,Регион,REPORT_DATE,BRANCH_ID,SAP_CODE,SAP_NAME_RU,PARAM_1,PARAM_3,PARAM_VALUE
0,Challenger Elite,Core Live Subs,233000,Tele2 - Vladimir,2021-01-01,28.0,233000,Tele2-Владимир,BU,Min,290024.0
1,Challenger wo SPB,Core Live Subs,240000,Tele2 - Kaluga,2021-01-01,31.0,240000,Tele2-Калуга,BU,Min,184342.0
2,Challenger wo SPB,Core Live Subs,244000,Tele2 - Kostroma,2021-01-01,32.0,244000,Tele2-Кострома,BU,Min,86525.0
3,Challenger wo SPB,Core Live Subs,262000,Tele2 - Ryazan,2021-01-01,39.0,262000,Tele2-Рязань,BU,Min,192061.0
4,Challenger Elite,Core Live Subs,267000,Tele2 - Smolensk,2021-01-01,11.0,267000,Tele2-Смоленск,BU,Min,197808.0


In [15]:
df.columns

Index(['Cluster', 'KPI', 'Код региона', 'Регион', 'REPORT_DATE', 'BRANCH_ID',
       'SAP_CODE', 'SAP_NAME_RU', 'PARAM_1', 'PARAM_3', 'PARAM_VALUE'],
      dtype='object')

In [16]:
df.columns = ['Cluster', 'PARAM_2', 'Код региона', 'Регион', 'REPORT_DATE', 'BRANCH_ID',
       'SAP_CODE', 'SAP_NAME_RU', 'PARAM_1', 'PARAM_3', 'PARAM_VALUE']

In [17]:
with teradatasql.connect() as session:
    query = '''
            SEL TOP 5 * from UAT_PRODUCT.PRODUCT_PARAMETERS
            '''
    pp = pd.read_sql(query, session)
pp.head()

,REPORT_DATE,PARAM_1,PARAM_2,PARAM_3,PARAM_4,BRANCH_ID,BASE_TYPE,TP_ID_1,TP_ID_2,TARIFF_1,TARIFF_2,PARAM_VALUE
0,2021-10-09,F1_2021_01,Gross Margin 1,None,None,14.0,NOT A,None,None,NO SUBS,None,2.018588e+06
1,2021-10-09,F1_2021_01,Average number of subscribers,None,None,14.0,NEW,None,None,PAYG,None,1.287065e+03
2,2021-10-09,BU,Number of Data Users,None,None,14.0,NEW,None,None,PAYG,None,3.691613e+02
3,2021-10-09,BU,Total Minutes technical,None,None,14.0,OLD,None,None,Bundle,None,3.209505e+07
4,2021-10-09,F1_2021_01,National MtL VC,None,None,14.0,NEW,None,None,Bundle,None,-6.233221e+04


In [18]:
df_old = df.copy()
df = df.reindex(columns = list(pp.columns),  fill_value = np.NaN).where((pd.notnull(df)), None)
df.head()

,REPORT_DATE,PARAM_1,PARAM_2,PARAM_3,PARAM_4,BRANCH_ID,BASE_TYPE,TP_ID_1,TP_ID_2,TARIFF_1,TARIFF_2,PARAM_VALUE
0,2021-01-01,BU,Core Live Subs,Min,None,28,None,None,None,None,None,290024
1,2021-01-01,BU,Core Live Subs,Min,None,31,None,None,None,None,None,184342
2,2021-01-01,BU,Core Live Subs,Min,None,32,None,None,None,None,None,86525
3,2021-01-01,BU,Core Live Subs,Min,None,39,None,None,None,None,None,192061
4,2021-01-01,BU,Core Live Subs,Min,None,11,None,None,None,None,None,197808


In [19]:
list(pp.columns)

['REPORT_DATE',
 'PARAM_1',
 'PARAM_2',
 'PARAM_3',
 'PARAM_4',
 'BRANCH_ID',
 'BASE_TYPE',
 'TP_ID_1',
 'TP_ID_2',
 'TARIFF_1',
 'TARIFF_2',
 'PARAM_VALUE']

In [20]:
df.head()

,REPORT_DATE,PARAM_1,PARAM_2,PARAM_3,PARAM_4,BRANCH_ID,BASE_TYPE,TP_ID_1,TP_ID_2,TARIFF_1,TARIFF_2,PARAM_VALUE
0,2021-01-01,BU,Core Live Subs,Min,None,28,None,None,None,None,None,290024
1,2021-01-01,BU,Core Live Subs,Min,None,31,None,None,None,None,None,184342
2,2021-01-01,BU,Core Live Subs,Min,None,32,None,None,None,None,None,86525
3,2021-01-01,BU,Core Live Subs,Min,None,39,None,None,None,None,None,192061
4,2021-01-01,BU,Core Live Subs,Min,None,11,None,None,None,None,None,197808


In [21]:
len(df)

189

In [22]:
def update_uat_pp_test(df, batchsize = 100000):
        with teradatasql.connect() as session:
            with session.cursor() as cur:
                
                print('deleting from uat_product.PRODUCT_PARAMETERS...')
                cur.execute('''
                    delete from uat_product.PRODUCT_PARAMETERS WHERE PARAM_1='BU' AND PARAM_2='Core Live Subs';
                ''')
                print(f'{cur.rowcount} rows deleted from uat_product.PRODUCT_PARAMETERS')

                print('inserting into UAT_PRODUCT.PRODUCT_PARAMETERS...')
                for num in range(0, len(df), batchsize):
                    cur.executemany(f'''
                     INSERT into UAT_PRODUCT.PRODUCT_PARAMETERS ({','.join('?'*len(df.columns))})
                    ''',
                        [tuple(row) for row in df.iloc[num:num+batchsize,:].itertuples(index=False)]
                        )
                print(f'{len(df)} rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS.')

In [23]:
update_uat_pp_test(df)

deleting from uat_product.PRODUCT_PARAMETERS...
0 rows deleted from uat_product.PRODUCT_PARAMETERS
inserting into UAT_PRODUCT.PRODUCT_PARAMETERS...
189 rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS.


In [ ]:
# newfilename = r'*.xlsx'
# phd_rev_branch.to_excel(newfilename, index = False)